<a href="https://colab.research.google.com/github/yurisalesc/langchain-gemini-notebooks/blob/main/chat_with_multi_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [ ]:
import google.generativeai as genai

In [ ]:
import textwrap

from dotenv import load_dotenv
from google.colab import userdata
from IPython.display import display, Markdown

In [ ]:
GOOGLE_API_KEY = userdata.get('GeminiProKey')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
!echo -e 'GOOGLE_API_KEY=' > .env

In [ ]:
!ls -a

In [ ]:
load_dotenv()

In [ ]:
def to_markdown(text):
    return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [ ]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [ ]:
%%time
response = model.generate_content("What are the mayor candidates for São Paulo, Brazil in 2024 elections?")

In [ ]:
response

In [ ]:
response.text

In [ ]:
to_markdown(response.text)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-pro')

In [ ]:
result = llm.invoke('What are the last 10 USA presidents?')

In [ ]:
to_markdown(result.content)

In [ ]:
import io
import numpy as np
import requests
from PIL import Image


def show_image(url):
    response = requests.get(url)
    image = Image.open(io.BytesIO(response.content))
    image.load()
    display(image)


show_image("https://images.foxtv.com/static.fox5dc.com/www.fox5dc.com/content/uploads/2024/09/764/432/trump-ai.jpg")

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Write a blog post based on this image",
        },
        {
            "type": "image_url",
            "image_url": "https://images.foxtv.com/static.fox5dc.com/www.fox5dc.com/content/uploads/2024/09/764/432/trump-ai.jpg"
        }
    ]
)

In [ ]:
llm.invoke([message])

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install chromadb

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
!mkdir pdfs

In [ ]:
!gdown "https://drive.google.com/uc?id=11cCngMwrOHFLbzHRfu6UTvWhFEAN17X3"

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load_and_split()

In [ ]:
print(data)

In [ ]:
print(data[3].page_content)

In [ ]:
context = "\n".join(
    str(p.page_content) for p in data
)

In [ ]:
print("The total number of words in the context:", len(context))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=100)
context = "\n\n".join(
    str(p.page_content) for p in data
)

In [ ]:
texts = text_splitter.split_text(context)

In [ ]:
print(len(texts))

In [ ]:
texts[1]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever()

In [ ]:
question = "Por que o não há calculo econômico no socialismo?"
docs = vector_index.invoke(question)

In [ ]:
docs

In [ ]:
prompt_template = """
    Responda a questão o mais detalhado possível a partir do contexto dado, tenha certeza de prover todos os detalhes.
    Se não detalhes suficientes no contexto provido, apenas responda "Resposta não disponível no contexto".
    Não responda com informações erradas.\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.8)

In [ ]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [ ]:
response = chain(
    {
        "input_documents": docs,
        "question": question
    },
    return_only_outputs=True
)

In [ ]:
response